# Decomposing image information
Stough, DIP

It can be valuable in compression to rethink of an image in terms of important and less important information. At the very core in this example, we recode each pair of pixels as a mean, difference. Then if the difference is quite small, it can potentially be ignored.

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import rescale

#Probably shouldn't have to repeat this in every script.
#https://matplotlib.org/users/dflt_style_changes.html
import matplotlib as mpl
mpl.rcParams['image.cmap'] = 'gray'

## A very simple basis for any pair of pixels.

In [2]:
phi = np.array([1, 1])/np.sqrt(2)
psi = np.array([1, -1])/np.sqrt(2)
H = np.stack((phi, psi), axis = 0)

In [3]:
#Decompose each row into a half-row of phi's and a half-row of psi's
#Assuming a multiple of 2 on the number of columns.
def haarDecompOnce(I):
    Ic = I.copy()
    for i in range(3):
        X = np.reshape(I[...,i].ravel(order='C'), (2, I[...,i].size//2), order='F')
        phis = np.inner(phi, X.transpose())
        psis = np.inner(psi, X.transpose())
#         print(psis)
        Ic[...,i] = np.concatenate([np.reshape(phis, (I.shape[0], I.shape[1]//2)),
                           np.reshape(psis, (I.shape[0], I.shape[1] // 2))],
                          axis=1)
    return Ic

def haarDecomp(I):
    if not np.all([np.log2(d).is_integer() for d in I.shape[:2]]):
        raise ValueError('haarDecomp: requires image with dimensions powers of 2.')

    howmany = min([int(np.log2(d)) for d in I.shape[:2]])

    J = I.copy()
    for h in range(howmany):
        J = haarDecompOnce(J)
        J = haarDecompOnce(J.transpose())
    return J

In [15]:
#Decompose each row into a half-row of phi's and a half-row of psi's
#Assuming a multiple of 2 on the number of columns.
def haarDecompOnce(I):
    X = np.reshape(I.ravel(order='C'), (2, I.size//2), order='F')
    print(X.shape)
    phis = np.inner(phi, X.transpose())
    psis = np.inner(psi, X.transpose())
    return np.concatenate([np.reshape(phis, (I.shape[0], I.shape[1]//2)),
                           np.reshape(psis, (I.shape[0], I.shape[1] // 2))],
                          axis=1)

def haarDecomp(I):
    if not np.all([np.log2(d).is_integer() for d in I.shape[:2]]):
        raise ValueError('haarDecomp: requires image with dimensions powers of 2.')

    howmany = min([int(np.log2(d)) for d in I.shape[:2]])

    J = I.copy()
    for h in range(howmany):
        J = haarDecompOnce(J)
        J = haarDecompOnce(J.transpose())
    return J

#Left for later...
def haarRecompOnce(J):
    f = np.inner(X.T, phi)
    s = np.inner(X.T, psi)

    IR = np.reshape(np.stack([f,s]).ravel(order='F'), J.shape)
    return IR

def haarRecomp(J):
    if not np.all([np.log2(d).is_integer() for d in J.shape[:2]]):
        raise ValueError('haarDecomp: requires image with dimensions powers of 2.')
    howmany = min([int(np.log2(d)) for d in J.shape[:2]])
    # smallest one
    I = J.copy()
    for i in range(howmany):
        I = haarRecompOnce(I.T)
        I = haarRecompOnce(I.T)
    return I

In [16]:
#Just going to do single channel here.
I = plt.imread('cat_small.png').astype('float')
GI = I.copy()[...,:3]
GI = 0.2989 * I[..., 0] + 0.5870 * I[..., 1] + 0.1140 * I[..., 2]
GI = GI/GI.max()

#Code to pad to the nearest power of two
dimsRounded = [int(np.power(2, np.ceil(np.log2(d)))) for d in GI.shape[:2]]

# Padded to the power of 2.
GIP = np.pad(GI, pad_width=((0, dimsRounded[0]-GI.shape[0]),
                            (0, dimsRounded[1]-GI.shape[1])),
             mode='constant', constant_values=0)



First = haarDecompOnce(GI)
First = First/First.max()
Second = haarDecompOnce(First.transpose()).transpose()


f, ax = plt.subplots(1,3, figsize=(10,3))

ax[0].imshow(GI)
ax[0].set_title('Original')

ax[1].imshow(First)
ax[1].set_title('Step 1: Row Transform')

ax[2].imshow(Second)
ax[2].set_title('Step 2: Column Transform')

plt.show()
plt.tight_layout()

(2, 131072)
(2, 131072)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plt.figure()
HI = haarDecomp(GIP)
plt.imshow(HI)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)
(2, 131072)


In [24]:
plt.figure()
plt.imshow(haarRecomp(HI))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
plt.close('all')

In [73]:
f, ax = plt.subplots(1,2, figsize=(10,3), sharex = False, sharey = True)
arr = np.array_split(First, 2, axis = 1)
ax[0].imshow(GIP)
ax[1].imshow((arr[0]+arr[1]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
J = haarDecompOnce(GIP)

(2, 131072)


In [11]:
X = np.stack([J[:, :J.shape[1]//2].ravel(), J[:, J.shape[1]//2:].ravel()]) # mean and change
P = np.inner(X.T, H)
np.reshape(P, (512,512))

array([[0.76592364, 0.78037995, 0.77801403, ..., 0.37883899, 0.37526834,
        0.37929847],
       [0.76351419, 0.75948406, 0.76588011, ..., 0.31290997, 0.32142971,
        0.3197655 ],
       [0.72200817, 0.75781985, 0.76184998, ..., 0.3197655 , 0.32333615,
        0.31930602],
       ...,
       [0.95869309, 0.96176073, 0.95819008, ..., 0.94336821, 0.94739834,
        0.95471335],
       [0.95869309, 0.95893533, 0.9577306 , ..., 0.93651267, 0.93530795,
        0.94336821],
       [0.95582416, 0.9549052 , 0.95893533, ..., 0.93011662, 0.93530795,
        0.93530795]])

In [10]:
GIP[:10]

array([[0.76592364, 0.78037995, 0.77801403, ..., 0.37883899, 0.37526834,
        0.37929847],
       [0.76351419, 0.75948406, 0.76588011, ..., 0.31290997, 0.32142971,
        0.3197655 ],
       [0.72200817, 0.75781985, 0.76184998, ..., 0.3197655 , 0.32333615,
        0.31930602],
       ...,
       [0.24564602, 0.43972541, 0.55751814, ..., 0.3169401 , 0.32051074,
        0.31648062],
       [0.15199056, 0.32026851, 0.46199974, ..., 0.31124576, 0.31930602,
        0.31124576],
       [0.08032367, 0.19200609, 0.37357914, ..., 0.31810129, 0.32213142,
        0.31689657]])

In [13]:
f = np.inner(X.T, phi)
s = np.inner(X.T, psi)

IR = np.reshape(np.stack([f,s]).ravel(order='F'), J.shape)

In [14]:
plt.figure()
plt.imshow(IR)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …